In [19]:
#Import
import pandas as pd
from pathlib import Path
from shapely.geometry import Point
import geopandas as gpd
import numpy as np

In [20]:
# Data Load
raw_path = Path(r"C:\Users\darre\traffic-risk-mapping\data\raw\NewYork_collisions_raw_20251110.csv")
df = pd.read_csv(raw_path, low_memory=False)

In [21]:
# Column Mapping
col_map = {
    'CRASH DATE': 'date',
    'CRASH TIME': 'time',
    'LATITUDE': 'latitude',
    'LONGITUDE': 'longitude',
    'ZIP CODE': 'zip_code',
    'LOCATION': 'location',
    'ON STREET NAME': 'street_name',
    'NUMBER OF PERSONS INJURED': 'number_of_persons_injured',
    'NUMBER OF PERSONS KILLED': 'number_of_persons_killed',
    'NUMBER OF PEDESTRIANS INJURED': 'number_of_pedestrians_injured',
    'NUMBER OF PEDESTRIANS KILLED': 'number_of_pedestrians_killed',
    'NUMBER OF CYCLIST INJURED': 'number_of_cyclist_injured',
    'NUMBER OF CYCLIST KILLED': 'number_of_cyclist_killed',
    'NUMBER OF MOTORIST INJURED': 'number_of_motorist_injured',
    'NUMBER OF MOTORIST KILLED': 'number_of_motorist_killed',
    'CONTRIBUTING FACTOR VEHICLE 1': 'vehicle_factor_1',
    'CONTRIBUTING FACTOR VEHICLE 2': 'vehicle_factor_2',
    'CONTRIBUTING FACTOR VEHICLE 3': 'vehicle_factor_3',
    'CONTRIBUTING FACTOR VEHICLE 4': 'vehicle_factor_4',
    'CONTRIBUTING FACTOR VEHICLE 5': 'vehicle_factor_5',
    'COLLISION_ID': 'collision_id',
    'VEHICLE TYPE CODE 1': 'vehicle_type_1',
    'VEHICLE TYPE CODE 2': 'vehicle_type_2',
    'VEHICLE TYPE CODE 3': 'vehicle_type_3',
    'VEHICLE TYPE CODE 4': 'vehicle_type_4',
    'VEHICLE TYPE CODE 5': 'vehicle_type_5',
}
df = df.rename(columns={k:v for k,v in col_map.items() if k in df.columns})

In [22]:
# Remove unnessecary columns
remove_cols = ['BOROUGH','CROSS STREET NAME', 'OFF STREET NAME']
df = df.drop(remove_cols, axis=1)

# Remove cols without contributing factor or street information
df = df.dropna(subset=['street_name','vehicle_factor_1'])

In [23]:
# Drop rows without coordinates
df = df.dropna(subset=['latitude', 'longitude'])

# Convert coordinates to numeric and drop invalids
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df = df.dropna(subset=['latitude','longitude'])

# Drop exact zeros
df = df[~((df['latitude'] == 0) | (df['longitude'] == 0))].copy()

# Remove rows outside geographics bounds for NYC
df = df[df['latitude'].between(40.3, 41.1) & df['longitude'].between(-74.5, -73.5)].copy()

# Combine date and time
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str), errors='coerce')
df = df.dropna(subset=['datetime'])

# Feature engineering
df['hour'] = df['datetime'].dt.hour
df['dayofweek'] = df['datetime'].dt.day_name()
df['is_weekend'] = df['dayofweek'].isin(['Saturday','Sunday'])

In [24]:
# GeoDataFrame conversion
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")
# check bounds
gdf.total_bounds

array([-74.25496 ,  40.498947, -73.66301 ,  41.061634])

In [25]:
# Export to cleaned csv
cleaned_csv = Path(r'C:\Users\darre\traffic-risk-mapping\data\processed\NYC_collisions_clean_20251110.csv')
gdf.to_csv(cleaned_csv, index=False)